In [8]:
def main():
    while True:
        try:
            r=requests.get(STATIONS,params={"apiKey":APIKEYS,"contract":NAME})
            store(json.loads(r.text))
            time.sleep(5*60)
        except:
            print(traceback.format_exc())
    return

In [1]:
import requests
import json
apiKey="9d55c6cd0e9961febb4a8d1430262600db3ddd3d"
Name="Dublin"
STATIONS="https://api.jcdecaux.com/vls/v1/stations"
r=requests.get(STATIONS,params={"apiKey":apiKey,"contract":Name})
json.loads(r.text)

[{'number': 42,
  'contract_name': 'dublin',
  'name': 'SMITHFIELD NORTH',
  'address': 'Smithfield North',
  'position': {'lat': 53.349562, 'lng': -6.278198},
  'banking': False,
  'bonus': False,
  'bike_stands': 30,
  'available_bike_stands': 4,
  'available_bikes': 26,
  'status': 'OPEN',
  'last_update': 1677615482000},
 {'number': 30,
  'contract_name': 'dublin',
  'name': 'PARNELL SQUARE NORTH',
  'address': 'Parnell Square North',
  'position': {'lat': 53.3537415547453, 'lng': -6.26530144781526},
  'banking': False,
  'bonus': False,
  'bike_stands': 20,
  'available_bike_stands': 12,
  'available_bikes': 8,
  'status': 'OPEN',
  'last_update': 1677615079000},
 {'number': 54,
  'contract_name': 'dublin',
  'name': 'CLONMEL STREET',
  'address': 'Clonmel Street',
  'position': {'lat': 53.336021, 'lng': -6.26298},
  'banking': False,
  'bonus': False,
  'bike_stands': 33,
  'available_bike_stands': 32,
  'available_bikes': 1,
  'status': 'OPEN',
  'last_update': 1677615294000},
 

In [2]:
import sqlalchemy as sqla
from sqlalchemy import create_engine
import traceback
import glob
import os
from pprint import pprint
import simplejson as json
import requests
import time
from IPython.display import display

In [3]:
URI="dbbikes.chdxsa0c6itr.eu-west-1.rds.amazonaws.com"
PORT=3306
DB="dbbikes"
USER="admin"
PASSWORD="xiaoxin123"
engine=create_engine("mysql+mysqldb://{}:{}@{}:{}/{}".format(USER,PASSWORD,URI,PORT,DB),echo=True)

In [7]:
sql="""
Create database if not exists dbbikes;
"""
engine.execute(sql)

2023-02-28 09:34:23,964 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-02-28 09:34:23,966 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-28 09:34:23,981 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-02-28 09:34:23,982 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-28 09:34:24,026 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-02-28 09:34:24,027 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-28 09:34:24,045 INFO sqlalchemy.engine.Engine 
Create database if not exists dbbikes;

2023-02-28 09:34:24,046 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-28 09:34:24,058 INFO sqlalchemy.engine.Engine COMMIT


In [10]:
for res in engine.execute("SHOW VARIABLES;"):
    print(res)

2023-02-28 09:36:43,550 INFO sqlalchemy.engine.Engine SHOW VARIABLES;
2023-02-28 09:36:43,551 INFO sqlalchemy.engine.Engine [raw sql] ()
('activate_all_roles_on_login', 'OFF')
('admin_address', '')
('admin_port', '33062')
('admin_ssl_ca', '')
('admin_ssl_capath', '')
('admin_ssl_cert', '')
('admin_ssl_cipher', '')
('admin_ssl_crl', '')
('admin_ssl_crlpath', '')
('admin_ssl_key', '')
('admin_tls_ciphersuites', '')
('admin_tls_version', 'TLSv1.2,TLSv1.3')
('authentication_policy', '*')
('auto_generate_certs', 'ON')
('auto_increment_increment', '1')
('auto_increment_offset', '1')
('autocommit', 'OFF')
('automatic_sp_privileges', 'ON')
('avoid_temporal_upgrade', 'OFF')
('back_log', '63')
('basedir', '/rdsdbbin/mysql-8.0.28.R4/')
('big_tables', 'OFF')
('bind_address', '*')
('binlog_cache_size', '32768')
('binlog_checksum', 'CRC32')
('binlog_direct_non_transactional_updates', 'OFF')
('binlog_encryption', 'OFF')
('binlog_error_action', 'ABORT_SERVER')
('binlog_expire_logs_seconds', '2592000')

In [11]:
# create table
sql="""
create table if not exists station(
address varchar(256),
banking integer,
bonus integer,
contract_name varchar(256),
name varchar(256),
number integer,
position_lat real,
position_lng real,
status varchar(256))
"""
try:
    res=engine.execute("Drop table if exists station")
    res=engine.execute(sql)
    print(res.fetchall())
except Exception as e:
    print(e)

2023-02-28 09:41:21,408 INFO sqlalchemy.engine.Engine Drop table if exists station
2023-02-28 09:41:21,411 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-28 09:41:21,438 INFO sqlalchemy.engine.Engine COMMIT
2023-02-28 09:41:21,458 INFO sqlalchemy.engine.Engine 
create table if not exists station(
address varchar(256),
banking integer,
bonus integer,
contract_name varchar(256),
name varchar(256),
number integer,
position_lat real,
position_lng real,
status varchar(256))

2023-02-28 09:41:21,460 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-28 09:41:21,702 INFO sqlalchemy.engine.Engine COMMIT
This result object does not return rows. It has been closed automatically.


In [13]:
sql="""
create table if not exists availability(
number integer,
available_bikes integer,
available_bikes_stands integer,
last_update integer)
"""
try:
    res=engine.execute(sql)
    print(res.fetchall())
except Exception as e:
    print(e)

2023-02-28 09:47:28,884 INFO sqlalchemy.engine.Engine 
create table if not exists availability(
number integer,
available_bikes integer,
available_bikes_stands integer,
last_update integer)

2023-02-28 09:47:28,887 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-28 09:47:29,057 INFO sqlalchemy.engine.Engine COMMIT
This result object does not return rows. It has been closed automatically.


In [ ]:
#crawling
import requests
import traceback
import datetime
import time
import json
apiKey="9d55c6cd0e9961febb4a8d1430262600db3ddd3d"
Name="Dublin"
STATIONS="https://api.jcdecaux.com/vls/v1/stations"
def write_to_file(r):
    now=datetime.datetime.now()
    with open("data/bikes_{}".format(now).replace(" ","_"),"w") as f:
        f.write(r.text)
def station_to_db(text):
    stations=json.loads(text)
    print(type(stations),len(stations))
    for station in stations:
        print(station)
        vals=(station.get("address"),int(station.get('banking')),int(station.get("bonus")),
             station.get("contract_name"),station.get("name"),station.get("number"),station.get("position").get("lat"),
              station.get("position").get("lng"),station.get("status"))
        engine.execute("insert into station values(%s,%s,%s,%s,%s,%s,%s,%s,%s)",vals)
        break
    return
def main():
    while True:
        try:
            now=datetime.datetime.now()
            r=requests.get(STATIONS,params={"apiKey":apiKey,"contract":Name})
            print(r,now)
            write_to_file(r)
            station_to_db(r.text)
            time.sleep(5*60)
        except:
            print(traceback.format_exc())
#             if engine is None:
    return
main()
    
            

<Response [200]> 2023-02-28 20:20:57.455446
<class 'list'> 114
{'number': 42, 'contract_name': 'dublin', 'name': 'SMITHFIELD NORTH', 'address': 'Smithfield North', 'position': {'lat': 53.349562, 'lng': -6.278198}, 'banking': False, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 4, 'available_bikes': 26, 'status': 'OPEN', 'last_update': 1677615482000}
2023-02-28 20:20:57,724 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-02-28 20:20:57,726 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-28 20:20:57,741 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-02-28 20:20:57,742 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-28 20:20:57,750 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-02-28 20:20:57,751 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-28 20:20:57,766 INFO sqlalchemy.engine.Engine insert into station values(%s,%s,%s,%s,%s,%s,%s,%s,%s)
2023-02-28 20:20:57,767 INFO sqlalchemy.engine.Engine [raw sql] ('Smithfield North', 0, 0, 'du

In [8]:
!cat data/bikes_2023-02-28_10\:44\:06.634456

[{"number":42,"contract_name":"dublin","name":"SMITHFIELD NORTH","address":"Smithfield North","position":{"lat":53.349562,"lng":-6.278198},"banking":false,"bonus":false,"bike_stands":30,"available_bike_stands":20,"available_bikes":9,"status":"OPEN","last_update":1677580967000},{"number":30,"contract_name":"dublin","name":"PARNELL SQUARE NORTH","address":"Parnell Square North","position":{"lat":53.3537415547453,"lng":-6.26530144781526},"banking":false,"bonus":false,"bike_stands":20,"available_bike_stands":19,"available_bikes":1,"status":"OPEN","last_update":1677580581000},{"number":54,"contract_name":"dublin","name":"CLONMEL STREET","address":"Clonmel Street","position":{"lat":53.336021,"lng":-6.26298},"banking":false,"bonus":false,"bike_stands":33,"available_bike_stands":24,"available_bikes":9,"status":"OPEN","last_update":1677580930000},{"number":108,"contract_name":"dublin","name":"AVONDALE ROAD","address":"Avondale Road","position":{"lat":53.359405,"lng":-6.276142},"banking":false,"